In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [3]:
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.maximize_window()

urls = []
base_url = "https://www.olx.com.br"

# Define o número de páginas a serem percorridas
num_paginas = 5  # Ajuste conforme necessário

for i in range(1, num_paginas + 1):
    url = f"{base_url}/imoveis/venda/casas?o={i}"
    driver.get(url)

    # Aguarda a página carregar completamente
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-ds-component='DS-NewAdCard-Link']"))
        )
    except Exception as e:
        print(f"Erro ao carregar a página {i}: {e}")
        continue

    # Faz scroll para carregar mais anúncios, se necessário
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tempo para novos elementos carregarem

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Se o scroll não carregou nada novo, para
        last_height = new_height

    # Coletar os links dos anúncios
    soup = BeautifulSoup(driver.page_source, "html.parser")
    anuncios = soup.find_all("a", {"data-ds-component": "DS-NewAdCard-Link"})

    for anuncio in anuncios:
        link = anuncio["href"]
        if link.startswith("/"):
            link = base_url + link  # Transforma link relativo em absoluto
        urls.append(link)

# Fechar o navegador Selenium
driver.quit()

# Exibir os links coletados
for link in urls:
    print(link)

print(f"Total de links coletados: {len(urls)}")

https://pr.olx.com.br/regiao-de-maringa/imoveis/casa-alto-padrao-em-porto-rico-pr-1224385124
https://pr.olx.com.br/regiao-de-maringa/imoveis/casa-alto-padrao-em-porto-rico-pr-1224385124
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/casa-residencial-comercial-a-300m-do-metro-1352433395
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/casa-residencial-comercial-a-300m-do-metro-1352433395
https://go.olx.com.br/grande-goiania-e-anapolis/imoveis/casa-semi-nova-impecavel-no-buena-vista-1367575344
https://go.olx.com.br/grande-goiania-e-anapolis/imoveis/casa-semi-nova-impecavel-no-buena-vista-1367575344
https://rj.olx.com.br/rio-de-janeiro-e-regiao/imoveis/casa-jardim-25-de-agosto-parcialmente-reformada-de-4-quartos-1276076221
https://rj.olx.com.br/rio-de-janeiro-e-regiao/imoveis/casa-jardim-25-de-agosto-parcialmente-reformada-de-4-quartos-1276076221
https://sp.olx.com.br/regiao-de-sao-jose-do-rio-preto/imoveis/imovel-para-aluguel-e-venda-possui-240-metros-quadrados-com-3-quartos-1391347499

In [4]:
urls = list(set(urls))
print(len(urls))

267


In [ ]:
df = pd.DataFrame()

# Funções auxiliares (mantidas como no seu código original)
def extrair_acabamentos(texto):
    padrao = r"porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado|tábua"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

def extrair_tipo_imovel(texto):
    padrao = r"casa|apartamento|sobrado|fazenda"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

scraper = cloudscraper.create_scraper()
urls = list(set(urls))  # Remove duplicatas

for url in urls:
    response = scraper.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        dados_imovel = {"Link": url}


        #Busca Localização
        location = soup.find("div", {"id": "location"})
        location_divs = location.find_all("div", recursive=False) if location else []
        for div in location_divs:
            texto = div.get_text(strip=True)
            texto = texto.replace("Localização", "").replace("Exibir no mapa", "").strip()
            cep = re.findall(r"\d+", texto)[-1]  # Captura o último número encontrado, que é o CEP
            # print(cep)

        #Busca Valor 
        div_valor_mae = soup.find("div", {"id": "price-box-container"})  # Encontra a div principal
        if div_valor_mae:
            div_valor = div_valor_mae.find("div", {"class": "olx-d-flex olx-fd-column"})  # Busca dentro dela
        else:
            print("Div com ID 'price-box-container' não encontrada")


        # Busca os dados do imóvel
        div_principal = soup.find("div", {"class": "ad__sc-wuor06-0 hfcCRQ"})
        divs_internas = div_principal.find_all("div", recursive=False) if div_principal else []
        for endereco, div in zip(location_divs ,divs_internas):
            end = endereco.get_text(" ",strip=True)
            cep = re.findall(r"\d+", end)[-1] 
            texto = div.get_text(strip=True)
            valor = div_valor.get_text(strip=True) if div_valor else None
            dados_imovel["Valor Imóvel"] = valor.replace("R$", "").replace(".", "").replace(",", ".").strip() if valor else None
            if "Área construída" in texto:
                dados_imovel["Área construída"] = re.search(r"(\d+)", texto).group(1)  # Captura apenas o número
            elif "Quartos" in texto:
                dados_imovel["Quartos"] = texto.replace("Quartos", "").strip()
            elif "Banheiros" in texto:
                dados_imovel["Banheiros"] = texto.replace("Banheiros", "").strip()
            elif "Vagas na garagem" in texto:
                dados_imovel["Vagas na garagem"] = texto.replace("Vagas na garagem", "").strip()
            dados_imovel["CEP"] = cep
            endereco_limpo = end.replace("Localização", "")\
                .replace("Exibir no mapa", "")\
                .replace(cep, "")\
                .strip()

            # Remove possíveis vírgulas ou hifens extras que possam ter ficado
            endereco_limpo = re.sub(r'[,\s-]+$', '', endereco_limpo).strip()

            dados_imovel["Endereço"] = endereco_limpo
            print(dados_imovel)


        #Capturar outros dados (descomente se necessário)
        descricao_div = soup.find("div", {"data-testid": "ad-description"})
        dados_imovel["Descrição"] = descricao_div.get_text(strip=True) if descricao_div else None

        endereco_div = soup.find("div", {"data-testid": "ad-location"})
        dados_imovel["Endereço"] = endereco_div.get_text(strip=True) if endereco_div else None

        preco_div = soup.find("h2", {"data-testid": "ad-price"})
        dados_imovel["Valor Imóvel"] = preco_div.get_text(strip=True) if preco_div else None

        dados_imovel["Acabamento"] = extrair_acabamentos(dados_imovel.get("Descrição", ""))
        dados_imovel["Tipo Imovel"] = extrair_tipo_imovel(dados_imovel.get("Descrição", ""))

        # Adicionar os dados ao DataFrame
        df = pd.concat([df, pd.DataFrame([dados_imovel])], ignore_index=True)
        
    else:
        print(f"Erro ao acessar o anúncio: {url} - Código {response.status_code}")

# Exibir os dados coletados
print(df)

In [13]:
df.to_excel("imoveis.xlsx", index=False)